# Basics
This page assumes you have Elastic and Kibana set up and are connected to the local host. Your user name will be elastic and your password, the generated password from terminal.

Here are the essential steps and concepts:

1. Elasticsearch Core Concepts:
   - Index: A container for documents of a single type.
   - Type: In 6.x, each index can have only one type.
   - Document: The basic unit of information in Elasticsearch, stored as JSON.
   - Node: A single server that is part of a cluster.
   - Cluster: A collection of one or more nodes.
   - Shards and Replicas: Mechanisms for distributing and replicating data across the cluster.

   Learning note: Understanding these core concepts is crucial for working with Elasticsearch effectively.

2. Kibana Console UI:
   - Use the Kibana Console UI for interacting with Elasticsearch via its REST API.
   - The Console provides auto-completion and formatting for queries.

   Learning note: The Console UI is a powerful tool for learning and testing Elasticsearch queries.

3. Basic CRUD Operations:
   - Create (Index): PUT /<index>/_doc/<id> or POST /<index>/_doc
   - Read (Get): GET /<index>/_doc/<id>
   - Update: POST /<index>/_update/<id>
   - Delete: DELETE /<index>/_doc/<id>

   Learning note: These operations form the foundation of interacting with Elasticsearch.

4. Mappings:
   - Mappings define how documents and their fields are stored and indexed.
   - Elasticsearch can infer mappings dynamically, but you can also define them explicitly.

   Learning note: Understanding and controlling mappings is crucial for optimizing search and analytics performance.

5. Creating Indices and Mappings:
   - Create an index: PUT /<index>
   - Create a mapping: PUT /<index>/_mapping/_doc

   Learning note: Explicitly creating indices and mappings gives you more control over your data structure.

6. Data Types:
   - Elasticsearch supports various data types like text, keyword, numeric types, date, boolean, etc.

   Learning note: Choosing the right data type for each field is important for efficient searching and storage.

7. REST API Overview:
   - Elasticsearch provides a comprehensive REST API for various operations.
   - Common API conventions include formatting responses and dealing with multiple indices.

   Learning note: Familiarize yourself with the API structure to effectively use Elasticsearch.

8. Searching:
   - Basic search: GET /<index>/_search
   - You can search across multiple indices and types.

   Learning note: Understanding how to construct search queries is fundamental to using Elasticsearch effectively.



## Examples

GET /


####  1. Creating an Index (Our Digital Wardrobe)
PUT /fashion_closet
{
  "settings": {
    "number_of_shards": 1,
    "number_of_replicas": 1
  }
}

#### The shards are like different sections of our wardrobe, and replicas are backup copies (because we never want to lose track of our favorite outfits)!


####  2. Adding Documents (Storing Outfits)
POST /fashion_closet/_doc
{
  "item": "Summer Dress",
  "color": "Pink",
  "occasion": "Beach Party",
  "brand": "Forever 21",
  "price": 29.99
}

#### 3. Retrieving Documents
GET /fashion_closet/_doc/n8wkxZAB1rYSz0fLWxOP

#### 4. Updating Documents
POST /fashion_closet/_update/n8wkxZAB1rYSz0fLWxOP
{
  "doc": {
    "accessories": ["Sunglasses", "Beach Bag", "HeadScarf"]
  }
}

#### 5. Deleting Documents
DELETE /fashion_closet/_doc/n8wkxZAB1rYSz0fLWxOP



#### 6. Searching 
GET /fashion_closet/_search
{
  "query": {
    "match": {
      "color": "Pink"
    }
  }
}

#### 7. Creating a Mapping (Organizing Our Closet)
PUT /fashion_closet/_mapping
{
  "properties": {
    "item": { "type": "text" },
    "color": { "type": "keyword" },
    "occasion": { "type": "text" },
    "brand": { "type": "keyword" },
    "price": { "type": "float" },
    "date_added": { "type": "date" }
  }
}

#### 8. Aggregations
GET /fashion_closet/_search
{
  "size": 0,
  "aggs": {
    "occasion_counts": {
      "terms": {
        "field": "occasion"
      }
    }
  }
}


## Common Problems

**Deleting and Searching Issue:**

When you delete an item and can still search for it, it's usually because of two reasons:
Sometimes, Elasticsearch takes a moment to update all its copies. Try this after deleting:
POST /fashion_closet/_refresh 
GET /fashion_closet/_doc/<id>
Here we have refreshed our index and double checked to make sure our id specific item is really gone.


**Mapping Text to Keyword Issue:**
 If you haven't added any data yet, you can delete the index and recreate it with the correct mapping:
 DELETE /fashion_closet

PUT /fashion_closet
{
  "mappings": {
    "properties": {
      "item": { "type": "text" },
      "color": { "type": "keyword" },
      "occasion": { "type": "keyword" },
      "brand": { "type": "keyword" },
      "price": { "type": "float" }
    }
  }
}
If you've already added data, you'll need to create a new index with the correct mapping, then reindex your data. It's like moving to a new, better-organized closet:
PUT /new_fashion_closet
{
  "mappings": {
    "properties": {
      "item": { "type": "text" },
      "color": { "type": "keyword" },
      "occasion": { "type": "keyword" },
      "brand": { "type": "keyword" },
      "price": { "type": "float" }
    }
  }
}

POST _reindex
{
  "source": {
    "index": "fashion_closet"
  },
  "dest": {
    "index": "new_fashion_closet"
  }
}
Then you can delete the old index and rename the new one if you want:
DELETE /fashion_closet

POST /_aliases
{
  "actions": [
    {
      "remove": {
        "index": "new_fashion_closet",
        "alias": "new_fashion_closet"
      }
    },
    {
      "add": {
        "index": "new_fashion_closet",
        "alias": "fashion_closet"
      }
    }
  ]
}

# Search


1. Analyzers:
   - Understand that analyzers process text fields for indexing and searching.
   - They consist of character filters, tokenizers, and token filters.
   - Elasticsearch has built-in analyzers, but you can create custom ones.

2. Query types:
   - Term-level queries: For exact matches on structured data (e.g., range, exists, term queries).
   - Full-text queries: For analyzed text fields (e.g., match, match_phrase, multi_match queries).
   - Compound queries: For combining multiple queries (e.g., bool, constant_score queries).

3. Query context vs. Filter context:
   - Query context: Calculates relevance scores.
   - Filter context: Just includes/excludes documents without scoring.

4. Bool query:
   - Combines multiple query clauses:
     - must: AND condition, contributes to score
     - should: OR condition, contributes to score
     - filter: AND condition, doesn't affect score
     - must_not: NOT condition, doesn't affect score

5. Relevance scoring:
   - Elasticsearch assigns relevance scores to matching documents.
   - You can boost fields or queries to influence scoring.

6. Basic query structure:
   - Understand the JSON structure for queries in Elasticsearch.

By grasping these core concepts, you'll have a solid foundation for understanding most of the chapter's content and working with Elasticsearch search functionality.

# Examples
## Update Index Mapping

PUT /fashion_closet
{
  "settings": {
    "analysis": {
      "analyzer": {
        "fashion_analyzer": {
          "type": "custom",
          "tokenizer": "standard",
          "char_filter": ["html_strip"],
          "filter": ["lowercase", "asciifolding", "fashion_synonym"]
        }
      },
      "filter": {
        "fashion_synonym": {
          "type": "synonym",
          "synonyms": [
            "dress, gown",
            "pants, trousers, slacks",
            "top, blouse, shirt"
          ]
        }
      }
    }
  },
  "mappings": {
    "properties": {
      "item": { 
        "type": "text",
        "analyzer": "fashion_analyzer"
      },
      "color": { "type": "keyword" },
      "occasion": { "type": "text" },
      "brand": { "type": "keyword" },
      "price": { "type": "float" },
      "date_added": { "type": "date" }
    }
  }
}

# Populate the Data
POST /fashion_closet/_doc
{
  "item": "Floral Summer Dress",
  "color": "Pink",
  "occasion": "Beach Party",
  "brand": "Forever 21",
  "price": 29.99,
  "date_added": "2023-07-18"
}

POST /fashion_closet/_doc
{
  "item": "Sequin Gown",
  "color": "Gold",
  "occasion": "Prom",
  "brand": "Sherri Hill",
  "price": 299.99,
  "date_added": "2023-07-17"
}

POST /fashion_closet/_doc
{
  "item": "Skinny Jeans",
  "color": "Blue",
  "occasion": "Casual",
  "brand": "Levi's",
  "price": 59.99,
  "date_added": "2023-07-16"
}

#### 3. Search using anaylzer
GET /fashion_closet/_search
{
  "query": {
    "match": {
      "item": "gown"
    }
  }
}

#### This should return both the "Floral Summer Dress" and "Sequin Gown" because of our synonyms!

#### 4. Bool query
GET /fashion_closet/_search
{
  "query": {
    "bool": {
      "must": [
        { "match": { "occasion": "Prom" } },
        { "range": { "price": { "gte": 200 } } }
      ],
      "should": [
        { "match": { "color": "Gold" } }
      ]
    }
  }
}


#### Shopping analytics
GET /fashion_closet/_search
{
  "size": 0,
  "aggs": {
    "color_palette": {
      "terms": {
        "field": "color",
        "size": 5
      }
    },
    "price_ranges": {
      "range": {
        "field": "price",
        "ranges": [
          { "to": 50 },
          { "from": 50, "to": 100 },
          { "from": 100, "to": 200 },
          { "from": 200 }
        ]
      }
    },
    "avg_price": {
      "avg": {
        "field": "price"
      }
    }
  }
}

# Analysis
Aggregations Overview:

Aggregations are used for analytics and data summarization in Elasticsearch.
Main types: Metric, Bucket, Matrix, and Pipeline aggregations.


Metric Aggregations:

Work on numeric fields to compute aggregate values like sum, average, min, max.
Examples: sum, avg, min, max, stats, extended_stats, cardinality.


Bucket Aggregations:

Segment data into buckets based on criteria.
Examples: terms, histogram, date_histogram, range, filters.


Nesting Aggregations:

Combine bucket and metric aggregations for complex analytics.


Date Histogram Aggregation:

Used for time-based analysis, slicing data into time intervals.


Geo Aggregations:

Analyze geographical data using geo_distance and geohash_grid.


Pipeline Aggregations:

Aggregate results of other aggregations.
Example: cumulative sum over time.